In [1]:
import scipy
import scipy.stats as sp
import scipy.optimize as spopt

import emcee
import batman
import corner

from astropy import constants as const
from astropy import units

import numpy as np
import time as t
import timeit
import os, sys
import csv

from multiprocessing import Pool
from threadpoolctl import threadpool_limits

import inspect

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable

import astropy.time
from astropy.stats import sigma_clip
from astropy.table import Table
from astropy.io import fits

import urllib.request

# SPCA libraries
from SPCA import helpers, astro_models, make_plots, make_plots_custom, detec_models, bliss

In [ ]:
planet = 'MASCARA-1b'
channel = 'ch2'
mode = 'Poly2_v1'
rootpath = '/home/taylor/Documents/Research/spitzer/MEGA/'

# parameters you do not wish to fit
dparams_input = []#['ecosw','esinw']

# parameters you want to place a gaussian prior on
gparams = ['t0', 'per', 'a', 'inc']

# parameters you want to place a uniform prior on
uparams = ['gpLx', 'gpLy']
uparams_limits = [[0,-3],[0,-3]]

ncpu = 3                                 # The number of cpu threads to be used when running MCMC
runMCMC = True                           # whether to run MCMC or just load-in past results
nBurnInSteps1 = 1e5                      # number of steps to use for the first mcmc burn-in (only used if not doing GP)
nBurnInSteps2 = 1e6                      # number of steps to use for the second mcmc burn-in
nProductionSteps = 2e5                   # number of steps to use with mcmc production run
usebestfit = False                       # used best-fit instead of most probable parameters 
blissNBin = 8                            # number of knots to allow in each direction
secondOrderOffset = False                # should you use the second order sinusoid terms when calculating offset
bestfitNbin = 50                         # the number of binned values to overplot on the bestfit 4-panel figure (use None if you don't want these overplotted)
nFrames  = 64                            # number of frames per binned data point
initializeWithOld = False                # initial with previous mcmc results using the same method
pldIgnoreFrames = True                   # Whether or not to use the PLD photometry that ignored bad frames
pldAddStack = False                      # Whether or not to use the PLD photometry that used background correction stacks
debug = False                            # True if user wants details about the lambda functions created

#non-unity if you have dilution by a nearby companion
compFactor = 1.

# non-zero if you want to remove some initial data points
cut_tmp = 0

In [ ]:
if rootpath[-1]!='/':
    rootpath += '/'

with open(rootpath+planet+'/analysis/'+channel+'/cutFirstAOR.txt', 'r') as file:
    cutFirstAOR = file.readline().strip()=='True'

#Download the most recent masterfile of the best data on each target    
dh.downloadExoplanetArchive()

# makes list of parameters that won't be fitted 
dparams = helpers.expand_dparams(dparams_input, mode)

p0_obj = dh.loadArchivalData(planet)

In [ ]:
# set up Gaussian priors
priors, errs = dh.setup_gpriors(gparams, p0_obj)

In [ ]:
(foldername, filename, filename_full, savepath,
 path_params, AOR_snip, aors, ignoreFrames) = dh.findPhotometry(rootpath, planet, channel,
                                                                mode, pldIgnoreFrames, pldAddStack)

breaks = dh.find_breaks()

sigF_photon_ppm = dh.get_photon_limit(foldername+filename, mode, nFrames, ignoreFrames)

gparams_unsorted = np.copy(gparams)
gparams = np.array([parm for parm in p0_obj.params if parm in gparams])

uparams_unsorted = np.copy(uparams)
uparams = np.array([parm for parm in p0_obj.params if parm in uparams])

In [ ]:
signalfunc, detecfunc = dh.get_detector_functions(mode)

## Load data

In [ ]:
# For datasets where the first AOR is peak-up data
if cutFirstAOR:
    rawfiles = np.sort(os.listdir(rootpath+planet+'/data/'+channel+'/'+aors[0]+'/'+channel+'/bcd/'))
    rawfiles  = [rawfile for rawfile in rawfiles if '_bcd.fits' in rawfile]
    cut = cut_tmp+len(rawfiles)
else:
    cut = cut_tmp

# loading full data set for BIC calculation afterwards
if 'pld' in mode.lower():
    # get data from unbinned photometry for chi2 on unbinned data calculation later
    P_full, flux_full, time_full = helpers.get_full_data(foldername+filename_full, mode,
                                                         cut=cut, nFrames=nFrames, ignore=ignoreFrames)
else:
    # get data from photometry
    (flux_full, fluxerr_full, time_full, xdata_full, ydata_full,
     psfxw_full, psfyw_full) = helpers.get_full_data(foldername+filename_full, mode, cut=cut,
                                                     nFrames=nFrames, ignore=ignoreFrames)

# Get Data we'll analyze
if 'pld' in mode.lower():
    # Get Data
    Pnorm_0, flux0, time0 = helpers.get_data(foldername+filename, mode)
    Pnorm, flux, time = helpers.get_data(foldername+filename, mode, cut=cut)
    
    # FIX: Add an initial PLD plot
    
else:
    # Get Data
    (flux0, flux_err0, time0, xdata0, ydata0,
     psfxw0, psfyw0) = helpers.get_data(foldername+filename, mode)
    (flux, flux_err, time, xdata, ydata,
     psfxw, psfyw) = helpers.get_data(foldername+filename, mode, cut=cut)

    ## FIX: peritime doesn't get made
    if True:#'ecosw' in dparams_input and 'esinw' in dparams_input:
        # make photometry plots
        make_plots.plot_photometry(time0, flux0, xdata0, ydata0, psfxw0, psfyw0, 
                        time, flux, xdata, ydata, psfxw, psfyw, breaks, savepath)
    else:
        # plot raw data
        make_plots.plot_photometry(time0, flux0, xdata0, ydata0, psfxw0, psfyw0, 
                        time, flux, xdata, ydata, psfxw, psfyw, breaks, savepath, peritime)

In [ ]:
# declare where the heaviside break occurs
if 'hside' in mode.lower():
    # FIX: This is fully broken right now
    p0_obj.s2 = timeaor1
    dparams = np.append(dparams, ['s2'])

# makes list of parameters that won't be fitted 
dparams = helpers.expand_dparams(dparams_input, mode)  

# if you want to use the best fit params from a previous MCMC run            
if initializeWithOld:
    dh.reload_old_fit(path_params, p0_obj)

In [ ]:
# get p0
p0, p0_labels, p0_fancyLabels = helpers.get_p0(dparams, p0_obj)

# make lambda functions to freeze certain inputs
astrofunc = helpers.make_lambdafunc(astro_models.ideal_lightcurve, dparams, p0_obj, debug=debug)
detecfunc = helpers.make_lambdafunc(detecfunc, dparams, p0_obj, debug=debug)
psfwifunc = helpers.make_lambdafunc(detec_models.detec_model_PSFW, dparams, p0_obj, debug=debug)
hsidefunc = helpers.make_lambdafunc(detec_models.hside, dparams, p0_obj, debug=debug)
tslopefunc = helpers.make_lambdafunc(detec_models.tslope, dparams, p0_obj, debug=debug)
lnpriorfunc = helpers.make_lambdafunc(helpers.lnprior, dparams, obj=p0_obj, debug=debug)

if gparams != [] or uparams != []:
    gpriorInds = [np.where(p0_labels==gpar)[0][0] for gpar in gparams]
    upriorInds = [np.where(p0_labels==upar)[0][0] for upar in uparams if upar in p0_labels]
    if 'gp' in mode.lower():
        gammaInd = np.where(p0_labels=='gpAmp')[0][0]
    else:
        gammaInd = None
    lnprior_custom = lambda p0: (helpers.lnprior_gaussian(p0, gpriorInds, priors, errs)+
                                 helpers.lnprior_uniform(p0, upriorInds, uparams_limits)+
                                 helpers.lnprior_gamma(p0, gammaInd, 1, 100))
else:
    lnprior_custom = None

# detemining which params in p0 are part of which functions
p0_astro  = helpers.get_fitted_params(astro_models.ideal_lightcurve, dparams)
p0_detec = helpers.get_fitted_params(detecfunc, dparams)
p0_psfwi  = helpers.get_fitted_params(detec_models.detec_model_PSFW)
p0_hside  = helpers.get_fitted_params(detec_models.hside)
p0_tslope  = helpers.get_fitted_params(detec_models.tslope)

# initial astro model
astro_guess = astrofunc(time, p0[np.where(np.in1d(p0_labels,p0_astro))])
resid       = flux/astro_guess

if 'bliss' in mode.lower():
    make_plots.plot_centroids(xdata0, ydata0, xdata, ydata, savepath)

    signal_inputs = bliss.precompute(flux, time, xdata, ydata, psfxw, psfyw, mode,
                                     astro_guess, blissNBin, savepath)
elif 'gp' in mode.lower():
    signal_inputs = [flux, time, xdata, ydata, psfxw, psfyw, mode]
    detec_inputs = [flux, xdata, ydata, time, True, astro_guess]
elif 'pld' in mode.lower():
    signal_inputs = [flux, time, Pnorm, mode]
    detec_inputs  = [Pnorm, mode]
elif 'poly' in mode.lower():
    signal_inputs = [flux, time, xdata, ydata, psfxw, psfyw, mode]
    detec_inputs = [xdata, ydata, mode]

# Run initial optimization

### If not GP, Run initial optimization on just the detector parameters

In [ ]:
# Run a first fit on the detector parameters to get into the right ballpark
if runMCMC:
    if not initializeWithOld and 'bliss' not in mode.lower() and 'gp' not in mode.lower():
        # Should work for PLD and Poly
        spyFunc0 = lambda p0_temp, inputs: np.mean((resid-detecfunc(inputs, *p0_temp))**2)
        spyResult0 = scipy.optimize.minimize(spyFunc0, p0[np.where(np.in1d(p0_labels,p0_detec))], detec_inputs, 'Nelder-Mead')

        # replace p0 with new detector coefficient values
        p0[np.where(np.in1d(p0_labels,p0_detec))] = spyResult0.x
        resid /= detecfunc(detec_inputs, *p0[np.where(np.in1d(p0_labels,p0_detec))])

        # 2) get initial guess for psfw model
        if 'psfw' in mode.lower():
            spyFunc0 = lambda p0_temp: np.mean((resid-psfwifunc([psfxw, psfyw], *p0_temp))**2)
            spyResult0 = scipy.optimize.minimize(spyFunc0, p0[np.where(np.in1d(p0_labels,p0_psfwi))], method='Nelder-Mead')

            # replace p0 with new detector coefficient values
            if spyResult0.success:
                p0[np.where(np.in1d(p0_labels,p0_psfwi))] = spyResult0.x
                resid /= psfwifunc([psfxw, psfyw], *p0[np.where(np.in1d(p0_labels,p0_psfwi))])

        # 3) get initial guess for hside model
        if 'hside' in mode.lower():
            spyFunc0 = lambda p0_temp: np.mean((resid-hsidefunc(time, *p0_temp))**2)
            spyResult0 = scipy.optimize.minimize(spyFunc0, p0[np.where(np.in1d(p0_labels,p0_hside))], method='Nelder-Mead')

            # replace p0 with new detector coefficient values
            if spyResult0.success:
                p0[np.where(np.in1d(p0_labels,p0_hside))] = spyResult0.x
                resid /= hsidefunc(time, *p0[np.where(np.in1d(p0_labels,p0_hside))])

        if 'tslope' in mode.lower():
            spyFunc0 = lambda p0_temp: np.mean((resid-tslopefunc(time, *p0_temp))**2)
            spyResult0 = scipy.optimize.minimize(spyFunc0, p0[np.where(np.in1d(p0_labels,p0_tslope))], method='Nelder-Mead')

            # replace p0 with new detector coefficient values
            if spyResult0.success:
                p0[np.where(np.in1d(p0_labels,p0_tslope))] = spyResult0.x
                resid /= tslopefunc(time, *p0[np.where(np.in1d(p0_labels,p0_tslope))])


        # initial guess
        signal_guess = signalfunc(signal_inputs, *p0)
        #includes psfw and/or hside functions if they're being fit
        detec_full_guess = signal_guess/astro_guess

In [ ]:
make_plots.plot_init_guess(time, flux, astro_guess, detec_full_guess)
plt.show()
plt.close()

### If GP, run initial full optimization to find best location

In [ ]:
if runMCMC and 'gp' in mode.lower():
    checkPhasePhis = np.linspace(-np.pi,np.pi,1000)

    initial_lnprob = helpers.lnprob(p0, signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom)

    spyFunc_full = lambda p0_temp, inputs: -helpers.lnprob(p0_temp, *inputs)

    nIterScipy = 10
    
    final_lnprob = -np.inf
    p0_optimized = []
    p0_temps = []
    print('Running iterative scipy.optimize')
    from tqdm import tqdm
    for i in tqdm(range(nIterScipy)):
        p0_rel_errs = 1e-1*np.ones_like(p0)
        gpriorInds = [np.where(p0_labels==gpar)[0][0] for gpar in gparams]
        p0_rel_errs[gpriorInds] = np.array(errs)/np.array(priors)
        p0_temp = p0*(1+p0_rel_errs*np.random.randn(len(p0)))+p0_rel_errs/10.*np.abs(np.random.randn(len(p0)))

        p0_temp[p0_labels=='A'] = np.random.uniform(0.,0.3)
        p0_temp[p0_labels=='B'] = np.random.uniform(-0.2,0.2)
        # Assignment to non-existent indices is safe (safelt ignores it), so this is fine for all modes
        p0_temp[p0_labels=='C'] = np.random.uniform(-0.3,0.3)
        p0_temp[p0_labels=='D'] = np.random.uniform(-0.3,0.3)
        p0_temp[p0_labels=='gpAmp'] = np.random.uniform(-4,-6)
        p0_temp[p0_labels=='gpLx'] = np.random.uniform(-0.5,-1)
        p0_temp[p0_labels=='gpLy'] = np.random.uniform(-0.5,-1)

        spyResult_full = scipy.optimize.minimize(spyFunc_full, p0_temp, [signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom], 'Nelder-Mead')
        lnprob_temp = helpers.lnprob(spyResult_full.x, signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom)

        p0_temps.append(np.copy(spyResult_full.x))

        if np.isfinite(lnprob_temp) and lnprob_temp > final_lnprob:
            final_lnprob = lnprob_temp
            p0_optimized = np.copy(spyResult_full.x)

            if final_lnprob > initial_lnprob:
                print('Improved ln-likelihood!')
                print("ln-likelihood: {0:.2f}".format(final_lnprob))
                p0 = np.copy(p0_optimized)

    astro_guess = astrofunc(time, *p0[np.where(np.in1d(p0_labels,p0_astro))])
    signal_guess = signalfunc(signal_inputs, *p0)
    #includes psfw and/or hside functions if they're being fit
    detec_full_guess = signal_guess/astro_guess

    # plot detector initial guess
    make_plots.plot_init_guess(time, flux, astro_guess, detec_full_guess)
    plt.show()
    plt.close()

### If GP, run an MCMC centred at the location of each optimization to break free of local minima

In [ ]:
if runMCMC and 'gp' in mode.lower():
    print('Running first burn-ins')
    p0_temps_mcmc = []
    for p0_temp in p0_temps:
        ndim = len(p0)
        nwalkers = ndim*3
        nBurnInSteps1 = 25500 # Chosen to give 500 steps per walker for Poly2v1 and 250 steps per walker for Poly5v2

        # get scattered starting point in parameter space 
        # MUST HAVE THE INITIAL SPREAD SUCH THAT EVERY SINGLE WALKER PASSES lnpriorfunc AND lnprior_custom
        p0_rel_errs = 1e-3*np.ones_like(p0_temp)
        gpriorInds = [np.where(p0_labels==gpar)[0][0] for gpar in gparams]
        p0_rel_errs[gpriorInds] = np.array(errs)/np.array(priors)
        pos0 = np.array([p0_temp*(1+p0_rel_errs*np.random.randn(ndim))+p0_rel_errs/10.*np.abs(np.random.randn(ndim)) for i in range(nwalkers)])

        checkPhasePhis = np.linspace(-np.pi,np.pi,1000)

        #sampler
        sampler = emcee.EnsembleSampler(nwalkers, ndim, helpers.lnprob, a = 2,
                                        args=(signalfunc, lnpriorfunc, 
                                              signal_inputs, checkPhasePhis, lnprior_custom))

        priorlnls = np.array([(lnpriorfunc(*p_tmp, mode, checkPhasePhis) != 0.0 or (lnprior_custom != 'none' and np.isinf(lnprior_custom(p_tmp)))) for p_tmp in pos0])
        iters = 10
        while np.any(priorlnls) and iters>0:
    #         print('Warning: Some of the initial values fail the lnprior!')
    #         print('Trying to re-draw positions...')
            p0_rel_errs /= 1.5
            pos0[priorlnls] = np.array([p0*(1+p0_rel_errs*np.random.randn(ndim))+p0_rel_errs/10.*np.abs(np.random.randn(ndim)) for i in range(np.sum(priorlnls))])
            priorlnls = np.array([(lnpriorfunc(*p_tmp, mode, checkPhasePhis) != 0.0 or (lnprior_custom != 'none' and np.isinf(lnprior_custom(p_tmp)))) for p_tmp in pos0])
            iters -= 1
        if iters==0 and np.any(priorlnls):
            print('Warning: Some of the initial values still fail the lnprior and the following MCMC will likely not work!')

        #Second burn-in
        #Do quick burn-in to get walkers spread out
        tic = t.time()
        pos1, prob, state = sampler.run_mcmc(pos0, np.rint(nBurnInSteps1/nwalkers), progress=False)
        print('Mean burn-in acceptance fraction: {0:.3f}'
                        .format(np.median(sampler.acceptance_fraction)))
        # sampler.reset()
        toc = t.time()
        print('MCMC runtime = %.2f min\n' % ((toc-tic)/60.))

        p0_temps_mcmc.append(np.copy(sampler.flatchain[np.argmax(sampler.flatlnprobability)]))

### If GP, run a final optimization

In [ ]:
if runMCMC and 'gp' in mode.lower():
    checkPhasePhis = np.linspace(-np.pi,np.pi,1000)

    initial_lnprob = helpers.lnprob(p0, signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom)

    spyFunc_full = lambda p0_temp, inputs: -helpers.lnprob(p0_temp, *inputs)

    final_lnprob = -np.inf
    p0_optimized = []
    p0_temps_final = []
    print('Running second iterative scipy.optimize')
    from tqdm import tqdm
    for p0_temp in tqdm(p0_temps_mcmc):

        spyResult_full = scipy.optimize.minimize(spyFunc_full, p0_temp, [signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom], 'Nelder-Mead')
        lnprob_temp = helpers.lnprob(spyResult_full.x, signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom)

        p0_temps_final.append(np.copy(spyResult_full.x))

        if np.isfinite(lnprob_temp) and lnprob_temp > final_lnprob:
            final_lnprob = lnprob_temp
            p0_optimized = np.copy(spyResult_full.x)

            if final_lnprob > initial_lnprob:
                print('Improved ln-likelihood!')
                print("ln-likelihood: {0:.2f}".format(final_lnprob))
                p0 = np.copy(p0_optimized)

    # if np.isfinite(final_lnprob) and final_lnprob > initial_lnprob:
    #     print('The full scipy optimize worked:')
    #     print("Initial ln-likelihood: {0:.2f}".format(initial_lnprob))
    #     print("Final ln-likelihood: {0:.2f}".format(final_lnprob))

    #     p0 = np.copy(p0_optimized)

    astro_guess = astrofunc(time, *p0[np.where(np.in1d(p0_labels,p0_astro))])
    signal_guess = signalfunc(signal_inputs, *p0)
    #includes psfw and/or hside functions if they're being fit
    detec_full_guess = signal_guess/astro_guess
    
    make_plots.plot_init_guess(time, flux, astro_guess, detec_full_guess)
    plt.show()
    plt.close()

## If not a GP, run a first MCMC burn-in

In [ ]:
if runMCMC and 'gp' not in mode.lower():
    ndim, nwalkers = len(p0), 150
    
    # get scattered starting point in parameter space 
    # MUST HAVE THE INITIAL SPREAD SUCH THAT EVERY SINGLE WALKER PASSES lnpriorfunc AND lnprior_custom
    p0_rel_errs = 1e-4*np.ones_like(p0)
    gpriorInds = [np.where(p0_labels==gpar)[0][0] for gpar in gparams]
    p0_rel_errs[gpriorInds] = np.array(errs)/np.array(priors)
    pos0 = np.array([p0*(1+p0_rel_errs*np.random.randn(ndim))+p0_rel_errs/10.*np.abs(np.random.randn(ndim)) for i in range(nwalkers)])

    checkPhasePhis = np.linspace(-np.pi,np.pi,1000)

    def templnprob(pars):
        return helpers.lnprob(pars, signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom)
    
    priorlnls = np.array([(lnpriorfunc(*p_tmp, mode, checkPhasePhis) != 0.0 or (lnprior_custom != 'none' and np.isinf(lnprior_custom(p_tmp)))) for p_tmp in pos0])
    iters = 10
    while np.any(priorlnls) and iters>0:
    #         print('Warning: Some of the initial values fail the lnprior!')
    #         print('Trying to re-draw positions...')
        p0_rel_errs /= 1.5
        pos0[priorlnls] = np.array([p0*(1+p0_rel_errs*np.random.randn(ndim))+p0_rel_errs/10.*np.abs(np.random.randn(ndim)) for i in range(np.sum(priorlnls))])
        priorlnls = np.array([(lnpriorfunc(*p_tmp, mode, checkPhasePhis) != 0.0 or (lnprior_custom != 'none' and np.isinf(lnprior_custom(p_tmp)))) for p_tmp in pos0])
        iters -= 1
    if iters==0 and np.any(priorlnls):
        print('Warning: Some of the initial values still fail the lnprior and the following MCMC will likely not work!')

        
        
    #First burn-in
    tic = t.time()
    print('Running first burn-in')
    with threadpool_limits(limits=1, user_api='blas'):
        with Pool(ncpu) as pool:
            #sampler
            sampler = emcee.EnsembleSampler(nwalkers, ndim, templnprob, a = 2, pool=pool)
            pos1, prob, state = sampler.run_mcmc(pos0, np.rint(nBurnInSteps1/nwalkers), progress=True)
    print('Mean burn-in acceptance fraction: {0:.3f}'
                .format(np.median(sampler.acceptance_fraction)))
    
    
    fname = savepath+'MCMC_'+mode+'_burnin1Walkers.pdf'
    helpers.walk_style(len(p0), nwalkers, sampler.chain, 10, int(np.rint(nBurnInSteps1/nwalkers)), p0_fancyLabels)
    plt.savefig(fname)
    plt.show()
    plt.close()
    
    
    p0 = sampler.flatchain[np.argmax(sampler.flatlnprobability)]
    astro_guess = astrofunc(time, *p0[np.where(np.in1d(p0_labels,p0_astro))])
    signal_guess = signalfunc(signal_inputs, *p0)
    #includes psfw and/or hside functions if they're being fit
    detec_full_guess = signal_guess/astro_guess
    fig = make_plots.plot_init_guess(time, flux, astro_guess, detec_full_guess)
    pathplot = savepath + '02_Initial_Guess.pdf'
    fig.savefig(pathplot, bbox_inches='tight')
    plt.show()
    plt.close()

## Run the MCMC

In [ ]:
if runMCMC:
    checkPhasePhis = np.linspace(-np.pi,np.pi,1000)

    number = int(1e3)
    with threadpool_limits(limits=1, user_api='blas'):
        avgRuntime = timeit.timeit(lambda: helpers.lnprob(p0, signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom), number=number)/float(number)
    estRuntime = avgRuntime*(nBurnInSteps2+nProductionSteps)/60.
    print('Estimated total MCMC runtime: ~'+str(int(np.rint(estRuntime/ncpu)))+' mins')

In [ ]:
ndim, nwalkers = len(p0), 150

if runMCMC:
    # get scattered starting point in parameter space 
    # MUST HAVE THE INITIAL SPREAD SUCH THAT EVERY SINGLE WALKER PASSES lnpriorfunc AND lnprior_custom
    p0_rel_errs = 1e-4*np.ones_like(p0)
    gpriorInds = [np.where(p0_labels==gpar)[0][0] for gpar in gparams]
    p0_rel_errs[gpriorInds] = np.array(errs)/np.array(priors)
    pos0 = np.array([p0*(1+p0_rel_errs*np.random.randn(ndim))+p0_rel_errs/10.*np.abs(np.random.randn(ndim)) for i in range(nwalkers)])

    checkPhasePhis = np.linspace(-np.pi,np.pi,1000)

    def templnprob(pars):
        return helpers.lnprob(pars, signalfunc, lnpriorfunc, signal_inputs, checkPhasePhis, lnprior_custom)

    priorlnls = np.array([(lnpriorfunc(*p_tmp, mode, checkPhasePhis) != 0.0 or (lnprior_custom != 'none' and np.isinf(lnprior_custom(p_tmp)))) for p_tmp in pos0])
    iters = 10
    while np.any(priorlnls) and iters>0:
    #         print('Warning: Some of the initial values fail the lnprior!')
    #         print('Trying to re-draw positions...')
        p0_rel_errs /= 1.5
        pos0[priorlnls] = np.array([p0*(1+p0_rel_errs*np.random.randn(ndim))+p0_rel_errs/10.*np.abs(np.random.randn(ndim)) for i in range(np.sum(priorlnls))])
        priorlnls = np.array([(lnpriorfunc(*p_tmp, mode, checkPhasePhis) != 0.0 or (lnprior_custom != 'none' and np.isinf(lnprior_custom(p_tmp)))) for p_tmp in pos0])
        iters -= 1
    if iters==0 and np.any(priorlnls):
        print('Warning: Some of the initial values still fail the lnprior and the following MCMC will likely not work!')

    #Second burn-in
    #Do quick burn-in to get walkers spread out
    tic = t.time()
    print('Running second burn-in')
    with threadpool_limits(limits=1, user_api='blas'):
        with Pool(ncpu) as pool:
            #sampler
            sampler = emcee.EnsembleSampler(nwalkers, ndim, templnprob, a = 2, pool=pool)
            pos1, prob, state = sampler.run_mcmc(pos0, np.rint(nBurnInSteps2/nwalkers), progress=True)
    print('Mean burn-in acceptance fraction: {0:.3f}'
                    .format(np.median(sampler.acceptance_fraction)))
    fname = savepath+'MCMC_'+mode+'_burninWalkers.pdf'
    helpers.walk_style(len(p0), nwalkers, sampler.chain, 10, int(np.rint(nBurnInSteps2/nwalkers)), p0_fancyLabels, fname)
    sampler.reset()
    toc = t.time()
    print('MCMC runtime = %.2f min\n' % ((toc-tic)/60.))


    #Run production
    #Run that will be saved
    tic = t.time()
    # Continue from last positions and run production
    print('Running production')
    with threadpool_limits(limits=1, user_api='blas'):
        with Pool(ncpu) as pool:
            #sampler
            sampler = emcee.EnsembleSampler(nwalkers, ndim, templnprob, a = 2, pool=pool)
            pos2, prob, state = sampler.run_mcmc(pos1, np.rint(nProductionSteps/nwalkers), progress=True)
    print("Mean acceptance fraction: {0:.3f}"
                    .format(np.mean(sampler.acceptance_fraction)))
    toc = t.time()
    print('MCMC runtime = %.2f min\n' % ((toc-tic)/60.))


    #Saving MCMC Results
    pathchain = savepath + 'samplerchain_'+mode+'.npy'
    pathposit = savepath + 'samplerposi_'+mode+'.npy'
    pathlnpro = savepath + 'samplerlnpr_'+mode+'.npy'
    np.save(pathchain, sampler.chain)
    np.save(pathposit, pos2)
    np.save(pathlnpro, prob)

    chain = sampler.chain
    
else:

    pathchain = savepath + 'samplerchain_'+mode+'.npy'
    chain = np.load(pathchain)
    pathlnpro = savepath + 'samplerlnpr_'+mode+'.npy'
    if os.path.exists(pathlnpro):
        lnprobability = np.load(pathlnpro)

samples = chain.reshape((-1, ndim))

## Output results from MCMC

In [ ]:
if 'inc' in p0_labels:
    pos_inc = np.where(p0_labels == 'inc')[0][0]
    samples[np.where(samples[:,pos_inc] > 90)[0],pos_inc] = 180 - samples[np.where(samples[:,pos_inc] > 90)[0],pos_inc]

In [ ]:
#print the results

(MCMC_Results) = np.array(list(map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84],axis=0)))))
p0_mcmc = np.median(samples, axis=0)

# taking max lnprob params instead of median bc degeneracy
if usebestfit == True: 
    if runMCMC == True:
        maxk, maxiter = np.unravel_index((sampler.lnprobability).argmax(), (sampler.lnprobability).shape)
        p0_mcmc = sampler.chain[maxk, maxiter,:]
    else:
        maxk, maxiter = np.unravel_index((lnprobability).argmax(), (lnprobability).shape)
        p0_mcmc = chain[maxk, maxiter,:]
    for i in range(len(p0_mcmc)):
        MCMC_Results[i] = (p0_mcmc[i], MCMC_Results[i][1], MCMC_Results[i][2])

# adjust fp, sigF, rp, r2 for dilution due to any nearby companion
if np.any(p0_labels == 'fp'):
    for i in range(3):
        MCMC_Results[np.where(p0_labels == 'fp')[0][0]][i] *= compFactor
if np.any(p0_labels == 'sigF'):
    for i in range(3):
        MCMC_Results[np.where(p0_labels == 'sigF')[0][0]][i] *= compFactor
if np.any(p0_labels == 'rp'):
    for i in range(3):
        MCMC_Results[np.where(p0_labels == 'rp')[0][0]][i] *= np.sqrt(compFactor)
if np.any(p0_labels == 'r2'):
    for i in range(3):
        MCMC_Results[np.where(p0_labels == 'r2')[0][0]][i] *= np.sqrt(compFactor)

In [ ]:
# printing output from MCMC
out = "MCMC result:\n\n"
for i in range(len(p0)):
    out += '{:>8} = {:>16}  +{:>16}  -{:>16}\n'.format(p0_labels[i],MCMC_Results[i][0], MCMC_Results[i][1], MCMC_Results[i][2])

# getting and printing the phase offset
As = samples[:,np.where(p0_labels == 'A')[0][0]][:,np.newaxis]
Bs = samples[:,np.where(p0_labels == 'B')[0][0]][:,np.newaxis]
phis = np.linspace(-np.pi,np.pi,1000)
offsets = []
stepSizeOffsets = int(1e2)
if ('A' in p0_labels)  and ('B' in p0_labels) and (('C' not in p0_labels and 'D' not in p0_labels) or not secondOrderOffset):
    for i in range(int(len(As)/stepSizeOffsets)):
        offsets.extend(-phis[np.argmax(1 + As[i*stepSizeOffsets:(i+1)*stepSizeOffsets]*(np.cos(phis)-1) + Bs[i*stepSizeOffsets:(i+1)*stepSizeOffsets]*np.sin(phis),axis=1)]*180/np.pi)
    if len(As)%stepSizeOffsets != 0:
        offsets.extend(-phis[np.argmax(1 + As[-len(As)%stepSizeOffsets:]*(np.cos(phis)-1) + Bs[-len(As)%stepSizeOffsets:]*np.sin(phis),axis=1)]*180/np.pi)
    offset = np.percentile(np.array(offsets), [16, 50, 84])[[1,2,0]]
    offset[1] -= offset[0]
    offset[2] = offset[0]-offset[2]
    out += '{:>8} = {:>16}  +{:>16}  -{:>16} degrees east\n'.format('Offset', offset[0], offset[1], offset[2])
elif ('A' in p0_labels)  and ('B' in p0_labels) and ('C' in p0_labels) and ('D' in p0_labels):
    Cs = samples[:,np.where(p0_labels == 'C')[0][0]][:,np.newaxis]
    Ds = samples[:,np.where(p0_labels == 'D')[0][0]][:,np.newaxis]
    for i in range(int(len(As)/stepSizeOffsets)):
        offsets.extend(-phis[np.argmax(1 + As[i*stepSizeOffsets:(i+1)*stepSizeOffsets]*(np.cos(phis)-1) + Bs[i*stepSizeOffsets:(i+1)*stepSizeOffsets]*np.sin(phis) + Cs[i*stepSizeOffsets:(i+1)*stepSizeOffsets]*(np.cos(2*phis)-1) + Ds[i*stepSizeOffsets:(i+1)*stepSizeOffsets]*np.sin(2*phis),axis=1)]*180/np.pi)
    if len(As)%stepSizeOffsets != 0:
        offsets.extend(-phis[np.argmax(1 + As[-len(As)%stepSizeOffsets:]*(np.cos(phis)-1) + Bs[-len(As)%stepSizeOffsets:]*np.sin(phis),axis=1)]*180/np.pi)
    offset = np.percentile(np.array(offsets), [16, 50, 84])[[1,2,0]]
    offset[1] -= offset[0]
    offset[2] = offset[0]-offset[2]
    out += '{:>8} = {:>16}  +{:>16}  -{:>16} degrees east\n'.format('Offset', offsets[0], offsets[1], offsets[2])

# print the R2/Rp ratio
if ('ellipse' in mode.lower()) and ('rp' in p0_labels) and ('r2' in p0_labels):
    out += '{:>8} = {:>16}\n'.format('R2/Rp', p0_mcmc[np.where(p0_labels == 'r2')[0][0]]/p0_mcmc[np.where(p0_labels == 'rp')[0][0]])

if channel == 'ch1':
    wav = 3.6*1e-6
elif channel == 'ch2':
    wav = 4.5*1e-6
if 'fp' in p0_labels:
    fp_MCMC = samples[:,np.where(p0_labels == 'fp')[0][0]]*compFactor
else:
    fp_MCMC = p0_obj['fp']
if 'rp' in p0_labels:
    rp_MCMC = samples[:,np.where(p0_labels == 'rp')[0][0]]*np.sqrt(compFactor)
else:
    rp_MCMC = p0_obj['rp']

tstar_bs = np.random.normal(p0_obj.tstar_b, p0_obj.tstar_b_err)

tday = const.h.value*const.c.value/(const.k_B.value*wav)*(np.log(1+(np.exp(const.h.value*const.c.value/(const.k_B.value*wav*tstar_bs))-1)/(fp_MCMC/rp_MCMC**2)))**-1
tnight = const.h.value*const.c.value/(const.k_B.value*wav)*(np.log(1+(np.exp(const.h.value*const.c.value/(const.k_B.value*wav*tstar_bs))-1)/(fp_MCMC*(1-2*As[:,0])/rp_MCMC**2)))**-1

out += '{:>8} = {:>16}  +{:>16}  -{:>16}\n'.format('T Day: ', np.median(tday), np.percentile(tday, 84)-np.median(tday), np.median(tday)-np.percentile(tday, 16))
out += '{:>8} = {:>16}  +{:>16}  -{:>16}\n'.format('T Night: ', np.nanmedian(tnight), np.nanpercentile(tnight, 84)-np.nanmedian(tnight), np.nanmedian(tnight)-np.nanpercentile(tnight, 16))
out += 'For T_{*,b} = '+str(tstar_b)+'\n'

print(out)
with open(savepath+'MCMC_RESULTS_'+mode+'.txt','w') as file:
    file.write(out) 

In [ ]:
ind_a = len(p0_astro) # index where the astro params end
labels = p0_fancyLabels[:ind_a]

fname = savepath+'MCMC_'+mode+'_astroWalkers.pdf'
helpers.walk_style(ind_a, nwalkers, chain, 10, chain.shape[1], labels, fname)

In [ ]:
if 'bliss' not in mode.lower() or 'sigF' not in dparams:
    labels = p0_fancyLabels[ind_a:]
    fname = savepath+'MCMC_'+mode+'_detecWalkers.pdf'
    helpers.walk_style(len(p0)-ind_a, nwalkers, chain[:,:,ind_a:], 10, chain.shape[1], labels, fname)

In [ ]:
if runMCMC:
    fig = corner.corner(samples[:,:ind_a], labels=p0_fancyLabels, quantiles=[0.16, 0.5, 0.84], show_titles=True, 
                        plot_datapoints=True, title_kwargs={"fontsize": 12})
    plotname = savepath + 'MCMC_'+mode+'_corner.pdf'
    fig.savefig(plotname, bbox_inches='tight')

In [ ]:
samples = None
sampler = None
chain = None

In [ ]:
# generate uniformly spaced time array for plot purposes
time2 = np.linspace(np.min(time), np.max(time), 1000)

# generate the models from best-fit parameters
mcmc_signal = signalfunc(signal_inputs, *p0_mcmc)
mcmc_lightcurve = astrofunc(time, *p0_mcmc[:ind_a])
mcmc_detec = mcmc_signal/mcmc_lightcurve

#for higher-rez red curve
mcmc_lightplot  = astrofunc(time2, *p0_mcmc[:ind_a])


# Differences from eccentricity
# if 'ecosw' in dparams and 'esinw' in dparams:
#     # converting time into orbital phases
#     if 't0' in p0_labels:
#         t0MCMC = p0_mcmc[np.where(p0_labels == 't0')[0][0]]
#     else:
#         t0MCMC = p0_obj.t0
#     if 'per' in p0_labels:
#         perMCMC = p0_mcmc[np.where(p0_labels == 'per')[0][0]]
#     else:
#         perMCMC = p0_obj.per
#     x = (time-t0MCMC)/perMCMC
#     orbNum = int(np.min(x))
#     if np.min(x)>0:
#         orbNum += 1
#     x -= orbNum
# 
#     orb_breaks = np.empty(len(breaks))
#     for j in range(len(breaks)):
#         orb_breaks[j] = ((breaks[j]-t0MCMC)/perMCMC-orbNum)      
# else:
#     x       = time - peritime
#     xbreaks = breaks - peritime

# FIX: peritime isn't defined, so just using time for all plots for now
orb_breaks = breaks
if True:#'ecosw' in dparams and 'esinw' in dparams:
    make_plots.plot_bestfit(time, flux, mcmc_lightcurve, mcmc_detec, mode, orb_breaks, savepath, nbin=bestfitNbin, fontsize=24)
    plt.close()
else:
    # FIX: make this default plotting option
    make_plots_custom.plot_bestfit(x, flux, mcmc_lightcurve, mcmc_detec, 
                                   mode, xbreaks, savepath, peritime=0, nbin=bestfitNbin)
    plt.close()


In [ ]:
# if McCubed is installed
try:
    from mc3.stats import time_avg
    intTime = (time[1]-time[0])
    minBins = 5
    residuals = flux/mcmc_detec - mcmc_lightcurve

    #WARNING: these durations assume circular orbits!!!
    ingrDuration = helpers.getIngressDuration(p0_mcmc, p0_labels, p0_obj, intTime)
    occDuration = helpers.getOccultationDuration(p0_mcmc, p0_labels, p0_obj, intTime)

    make_plots.plot_rednoise(residuals, minBins, ingrDuration, occDuration, intTime, mode, savepath, savetxt=True)
    plt.close()
except ImportError:
    #Noise vs bin-size to look for red noise
    residuals = flux/mcmc_detec - mcmc_lightcurve

    sigmas = []
    for i in range(3,len(residuals)):
        sigmas.append(helpers.binnedNoise(time,residuals,i))
    sigmas = np.array(sigmas)

    n_binned = len(residuals)/np.arange(3,len(residuals))

    #In case there is a NaN or something while binning
    n_binned = n_binned[np.where(np.isfinite(sigmas))[0]]
    sigmas = sigmas[np.where(np.isfinite(sigmas))[0]]


    ax = plt.gca()
    ax.set_yscale('log')
    ax.set_xscale('log')

    if 'sigF' in p0_labels:
        sigFMCMC = p0_mcmc[np.where(p0_labels == 'sigF')[0][0]]
    else:
        sigFMCMC = p0_obj.sigF
    if 'rp' in p0_labels:
        rpMCMC = p0_mcmc[np.where(p0_labels == 'rp')[0][0]]
    else:
        rpMCMC = p0_obj.rp
    if 'a' in p0_labels:
        aMCMC = p0_mcmc[np.where(p0_labels == 'a')[0][0]]
    else:
        aMCMC = p0_obj.a
    if 'per' in p0_labels:
        perMCMC = p0_mcmc[np.where(p0_labels == 'per')[0][0]]
    else:
        perMCMC = p0_obj.per

    #FIX: WARNING: these durations assume circular orbits!!!
    eclDuration = (2*rpMCMC/(2*np.pi*aMCMC/perMCMC))/((time[1]-time[0])) #Eclipse/transit ingress time
    trDuration = (2/(2*np.pi*aMCMC/perMCMC))/((time[1]-time[0])) #Transit/eclipse duration

    ax.plot(n_binned, sigmas, c='black', label='Data')
    ax.plot([n_binned[-1],n_binned[0]], [sigFMCMC, sigFMCMC/np.sqrt(n_binned[0])], c='red', label='White Noise')
    ylim = ax.get_ylim()
    plt.plot([eclDuration,eclDuration],ylim, color='black', ls='--', alpha=0.6)
    plt.plot([trDuration,trDuration],ylim, color='black', ls='-.', alpha=0.6)
    ax.set_ylim(ylim)
    plt.ylabel(r'$\sigma$ (ppm)', fontsize='x-large')
    plt.xlabel(r'N$_{\rm binned}$', fontsize='x-large')
    plt.legend(loc='best', fontsize='large')
    plotname = savepath + 'MCMC_'+mode+'_RedNoise.pdf'
    plt.savefig(plotname, bbox_inches='tight')
    plt.show()
    plt.close()


    #Figure out how much red noise we have

    #Eclipse Duration
    sreal = sigmas[np.where(n_binned<=eclDuration)[0][0]]*1e6
    s0 = sigFMCMC/np.sqrt(n_binned[np.where(n_binned<=eclDuration)[0][0]])*1e6
    outStr = 'Over Ingress ('+str(round(eclDuration*((time[1]-time[0]))*24*60, 1))+' min):\n'
    outStr += 'Expected Noise (ppm)\t'+'Observed Noise (ppm)\n'
    outStr += str(s0)+'\t'+str(sreal)+'\n'
    outStr += 'Observed/Expected\n'
    outStr += str(sreal/s0)+'\n\n'
    #Transit Duration
    sreal = sigmas[np.where(n_binned<=trDuration)[0][0]]*1e6
    s0 = sigFMCMC/np.sqrt(n_binned[np.where(n_binned<=trDuration)[0][0]])*1e6
    outStr += 'Over Transit/Eclipse ('+str(round(trDuration*((time[1]-time[0]))*24*60, 1))+' min):\n'
    outStr += 'Expected Noise (ppm)\t'+'Observed Noise (ppm)\n'
    outStr += str(s0)+'\t'+str(sreal)+'\n'
    outStr += 'Observed/Expected\n'
    outStr += str(sreal/s0)

    print(outStr)
    with open(plotname[:-3]+'txt','w') as file:
        file.write(outStr)

In [ ]:
#Binned data
data = flux/mcmc_detec
astro  = mcmc_lightcurve
if 'sigF' in p0_labels:
    sigFMCMC = p0_mcmc[np.where(p0_labels == 'sigF')[0][0]]
else:
    sigFMCMC = p0_obj.sigF
if 'bliss' in mode.lower():
    nKnotsUsed = len(signal_inputs[-4][signal_inputs[-2]])
    ndim_eff = ndim+nKnotsUsed
else:
    ndim_eff = ndim
chisB = helpers.chi2(data, astro, sigFMCMC)
logLB = helpers.loglikelihood(data, astro, sigFMCMC)
EB = helpers.evidence(logLB, ndim, len(data))
BICB = -2*EB

out = """Binned data:
chi2 = {0}
chi2datum = {1}
Likelihood = {2}
Evidence = {3}
BIC = {4}""".format(chisB, chisB/len(flux), logLB, EB, BICB)

if 'gp' not in mode.lower():
    #Unbinned data
    '''Get model'''
    astro_full   = astrofunc(time_full, *p0_mcmc[:ind_a])
    if 'bliss' in mode.lower():
        signal_inputs_full = bliss.precompute(flux_full, time_full, xdata_full, ydata_full,
                                              psfxw_full, psfyw_full, mode,
                                              astro_full, blissNBin, savepath, False)
    elif 'pld' in mode.lower():
        signal_inputs_full = (flux_full, time_full, Pnorm_full, mode)
        detec_inputs  = (Pnorm, mode)
    elif 'poly' in mode.lower():# and 'psfw' in mode.lower():
        signal_inputs_full = (flux_full, time_full, xdata_full, ydata_full, psfxw_full, psfyw_full, mode)

    signal_full = signalfunc(signal_inputs_full, *p0_mcmc)
    detec_full = signal_full/astro_full
    data_full = flux_full/detec_full

    '''Get Fitted Uncertainty'''
    ferr_full = sigFMCMC*np.sqrt(nFrames)

    N = len(data_full)
    if 'bliss' in mode.lower():
        nKnotsUsed_full = len(signal_inputs_full[-4][signal_inputs_full[-2]])
        ndim_eff_full = ndim+nKnotsUsed_full
    else:
        ndim_eff_full = ndim

    chis = helpers.chi2(data_full, astro_full, ferr_full)
    logL = helpers.loglikelihood(data_full, astro_full, ferr_full)
    E = helpers.evidence(logL, ndim_eff_full, N)
    BIC = -2*E

    out += """

Unbinned data:
chi2 = {0}
chi2datum = {1}
Likelihood = {2}
Evidence = {3}
BIC = {4}""".format(chis, chis/len(flux_full), logL, E, BIC)

with open(savepath+'EVIDENCE_'+mode+'.txt','w') as file:
    file.write(out)
print(out)

In [ ]:
ResultMCMC_Params = Table()

for i in range(len(p0_labels)):
    ResultMCMC_Params[p0_labels[i]] = MCMC_Results[i]

ResultMCMC_Params['offset'] = offset
ResultMCMC_Params['tDay'] = [np.nanmedian(tday), np.nanpercentile(tday, 84)-np.nanmedian(tday), np.nanmedian(tday)-np.nanpercentile(tday, 16)]
ResultMCMC_Params['tNight'] = [np.nanmedian(tnight), np.nanpercentile(tnight, 84)-np.nanmedian(tnight), np.nanmedian(tnight)-np.nanpercentile(tnight, 16)]

ResultMCMC_Params['chi2B'] = [chisB]
ResultMCMC_Params['chi2datum'] = [chisB/len(flux)]
ResultMCMC_Params['logLB'] = [logLB]
ResultMCMC_Params['evidenceB'] = [EB]
ResultMCMC_Params['sigF_photon_ppm'] = [sigF_photon_ppm]

if 'gp' not in mode.lower():
    ResultMCMC_Params['chi2'] = [chis]
    ResultMCMC_Params['logL'] = [logL]
    ResultMCMC_Params['evidence'] = [E]

pathres = savepath + 'ResultMCMC_'+mode+'_Params.npy'
np.save(pathres, ResultMCMC_Params)

In [ ]:
if 'pld' not in mode.lower():
    # determining in-eclipse and in-transit index

    # generating transit model

    if 't0' in p0_labels:
        t0MCMC = p0_mcmc[np.where(p0_labels == 't0')[0][0]]
    else:
        t0MCMC = p0_obj['t0']
    if 'per' in p0_labels:
        perMCMC = p0_mcmc[np.where(p0_labels == 'per')[0][0]]
    else:
        perMCMC = p0_obj['per']
    if 'rp' in p0_labels:
        rpMCMC = p0_mcmc[np.where(p0_labels == 'rp')[0][0]]
    else:
        rpMCMC = p0_obj['rp']
    if 'a' in p0_labels:
        aMCMC = p0_mcmc[np.where(p0_labels == 'a')[0][0]]
    else:
        aMCMC = p0_obj['a']
    if 'inc' in p0_labels:
        incMCMC = p0_mcmc[np.where(p0_labels == 'inc')[0][0]]
    else:
        incMCMC = p0_obj['inc']
    if 'ecosw' in p0_labels:
        ecoswMCMC = p0_mcmc[np.where(p0_labels == 'ecosw')[0][0]]
    else:
        ecoswMCMC = p0_obj['ecosw']
    if 'esinw' in p0_labels:
        esinwMCMC = p0_mcmc[np.where(p0_labels == 'esinw')[0][0]]
    else:
        esinwMCMC = p0_obj['esinw']
    if 'q1' in p0_labels:
        q1MCMC = p0_mcmc[np.where(p0_labels == 'q1')[0][0]]
    else:
        q1MCMC = p0_obj['q1']
    if 'q2' in p0_labels:
        q2MCMC = p0_mcmc[np.where(p0_labels == 'q2')[0][0]]
    else:
        q2MCMC = p0_obj['q2']
    if 'fp'in p0_labels:
        fpMCMC = p0_mcmc[np.where(p0_labels == 'fp')[0][0]]
    else:
        fpMCMC = p0_obj['fp']

    eccMCMC = np.sqrt(ecoswMCMC**2 + esinwMCMC**2)
    wMCMC   = np.arctan2(esinwMCMC, ecoswMCMC)
    u1MCMC  = 2*np.sqrt(q1MCMC)*q2MCMC
    u2MCMC  = np.sqrt(q1MCMC)*(1-2*q2MCMC)

    trans, t_sec, true_anom = astro_models.transit_model(time, t0MCMC, perMCMC, rpMCMC,
                                                         aMCMC, incMCMC, eccMCMC, wMCMC,
                                                         u1MCMC, u2MCMC)
    # generating secondary eclipses model
    eclip = astro_models.eclipse(time, t0MCMC, perMCMC, rpMCMC, aMCMC, incMCMC, eccMCMC, wMCMC,
                                 fpMCMC, t_sec)

    # get in-transit indices
    ind_trans  = np.where(trans!=1)
    # get in-eclipse indices
    ind_eclip  = np.where((eclip!=(1+fpMCMC)))
    # seperating first and second eclipse
    ind_ecli1 = ind_eclip[0][np.where(ind_eclip[0]<int(len(time)/2))]
    ind_ecli2 = ind_eclip[0][np.where(ind_eclip[0]>int(len(time)/2))]
    
    
    
    residual = flux/mcmc_detec - mcmc_lightcurve

    data1 = [xdata, ydata, psfxw, psfyw, flux, residual]
    data2 = [xdata[ind_ecli1], ydata[ind_ecli1], psfxw[ind_ecli1], psfyw[ind_ecli1], flux[ind_ecli1], residual[ind_ecli1]]
    data3 = [xdata[ind_trans], ydata[ind_trans], psfxw[ind_trans], psfyw[ind_trans], flux[ind_trans], residual[ind_trans]]
    data4 = [xdata[ind_ecli2], ydata[ind_ecli2], psfxw[ind_ecli2], psfyw[ind_ecli2], flux[ind_ecli2], residual[ind_ecli2]]

    plotname = savepath + 'MCMC_'+mode+'_7.pdf'
    make_plots.triangle_colors(data1, data2, data3, data4, plotname)